# Import & Ingest

In [2143]:
import sys
import pathlib
SOURCE_PATH = pathlib.Path.cwd().resolve().parent
sys.path.append(str(SOURCE_PATH))

In [2144]:
from config import *
from utils import ingest_kaggle_database
import xgboost
from sklearn.linear_model import LogisticRegression
import joblib
import pandas as pd
import sqlite3

In [2145]:
ingest_kaggle_database(SOCCER_DATABASE)

In [2146]:
%load_ext sql
%sql sqlite:///../data/database.sqlite
%config SqlMagic.style = "_deprecated_default"

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2147]:
%%sql
SELECT name FROM sqlite_schema
WHERE type = "table"
AND name NOT LIKE "sqlite%"
ORDER BY 1;

 * sqlite:///../data/database.sqlite
Done.


name
Country
League
Match
Player
Player_Attributes
Team
Team_Attributes


# Relevant Metadata

In [2148]:
%%sql
PRAGMA table_info(Country)

 * sqlite:///../data/database.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,name,TEXT,0,None,0


In [2149]:
%%sql
PRAGMA table_info(League)

 * sqlite:///../data/database.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,country_id,INTEGER,0,None,0
2,name,TEXT,0,None,0


In [2150]:
%%sql
PRAGMA table_info(Match)

 * sqlite:///../data/database.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,country_id,INTEGER,0,None,0
2,league_id,INTEGER,0,None,0
3,season,TEXT,0,None,0
4,stage,INTEGER,0,None,0
5,date,TEXT,0,None,0
6,match_api_id,INTEGER,0,None,0
7,home_team_api_id,INTEGER,0,None,0
8,away_team_api_id,INTEGER,0,None,0
9,home_team_goal,INTEGER,0,None,0


In [2151]:
%%sql
PRAGMA table_info(Team_Attributes)

 * sqlite:///../data/database.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,team_fifa_api_id,INTEGER,0,None,0
2,team_api_id,INTEGER,0,None,0
3,date,TEXT,0,None,0
4,buildUpPlaySpeed,INTEGER,0,None,0
5,buildUpPlaySpeedClass,TEXT,0,None,0
6,buildUpPlayDribbling,INTEGER,0,None,0
7,buildUpPlayDribblingClass,TEXT,0,None,0
8,buildUpPlayPassing,INTEGER,0,None,0
9,buildUpPlayPassingClass,TEXT,0,None,0


In [2152]:
%%sql
PRAGMA table_info(Player_Attributes)

 * sqlite:///../data/database.sqlite
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,player_fifa_api_id,INTEGER,0,None,0
2,player_api_id,INTEGER,0,None,0
3,date,TEXT,0,None,0
4,overall_rating,INTEGER,0,None,0
5,potential,INTEGER,0,None,0
6,preferred_foot,TEXT,0,None,0
7,attacking_work_rate,TEXT,0,None,0
8,defensive_work_rate,TEXT,0,None,0
9,crossing,INTEGER,0,None,0


# Joining Team Attributes

- Need: Match DataFrame with relevant Match columns plus the difference between the teams' respective stats, with both sets of stats being the measurements closest to that date for that team BEFORE the date of the match; matches earlier than first stats for the majority of teams will be dropped; dropping teams entirely whose stats come exceptionally late.
- Have: Match table with home_team_api_id, away_team_api_id, and date, plus Team_Attributes table with team_api_id
and repeated measurements of all team stats across several different dates.
- Steps: (I need a better understanding of SQL and pandas to conceptualize how to proceed, and what to use for what, and in what order)
    - Can I do SQL-like joins once the data are in pandas dataframes (including joining team attributes for only 288 teams to thousands of matches)?
    - What feature engineering and filtering can I do in SQL?
    - Can I progressively manipulate tables in SQL like I can in pandas, or does everything in SQL (joins, comparative aggregations or differences) have to be within one master query?

(SQL JOINs seem to unavoidably mess up order -- is that so, and does it mean I should use pandas for ordering before using pandas to also do the final joining?)

- Ideas:
    - Get home team match table using SQL
    - Get away team stats using SQL

## First Attempt

In [2153]:
%%sql
id << SELECT id FROM Match;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable id


In [2154]:
%%sql
match_api_id << SELECT match_api_id FROM Match;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable match_api_id


In [2155]:
#Number of unique matches? Any duplicates?
id = id.DataFrame()
match_api_id = match_api_id.DataFrame()
len(set(id.id)), len(set(match_api_id.match_api_id))

(25979, 25979)

In [2156]:
%%sql
df_home << WITH MatchCols AS (SELECT country_id, league_id, season, stage, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal FROM Match)                     
SELECT * FROM MatchCols m INNER JOIN Country c on m.country_id = c.id
INNER JOIN League l ON l.id = m.league_id
INNER JOIN Team_Attributes ta ON ta.team_api_id = m.home_team_api_id;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable df_home


In [2157]:
%%sql
df_away << WITH AwayID AS (SELECT home_team_api_id, away_team_api_id FROM Match)
SELECT * FROM AwayID a INNER JOIN Team_Attributes ta
ON a.away_team_api_id = ta.team_api_id;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable df_away


In [2158]:
df_home = df_home.DataFrame()
df_away = df_away.DataFrame()

df_home.shape, df_away.shape

((142093, 40), (142100, 27))

In [2159]:
df_home[["home_team_api_id", "away_team_api_id"]].head()

,home_team_api_id,away_team_api_id
0,9987,9993
1,9987,9993
2,9987,9993
3,9987,9993
4,9987,9993


In [2160]:
#df_home and df_away don't match in their alignment of the home teams and away teams; something is going on.
df_away[["home_team_api_id", "away_team_api_id"]].head()

,home_team_api_id,away_team_api_id
0,9987,9993
1,9987,9993
2,9987,9993
3,9987,9993
4,10000,9994


In [2161]:
#Between the two DataFrames, the away team only matches a fraction of the time
int((df_home.away_team_api_id == df_away.away_team_api_id.iloc[:len(df_home)]).sum()), len(df_home)

(68522, 142093)

In [2162]:
%%sql
df_ta << SELECT * FROM Team_Attributes;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable df_ta


In [2163]:
df_ta = df_ta.DataFrame()

In [2164]:
df_ta.shape

(1458, 25)

#### Dates

In [2165]:
%%sql
df_match << SELECT * FROM Match;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable df_match


In [2166]:
df_match = df_match.DataFrame()

In [2167]:
df_match.date = pd.to_datetime(df_match.date)
df_match.date.describe()

count                            25979
mean     2012-06-30 17:53:53.334616320
min                2008-07-18 00:00:00
25%                2010-05-09 00:00:00
50%                2012-05-13 00:00:00
75%                2014-08-17 00:00:00
max                2016-05-25 00:00:00
Name: date, dtype: object

In [2168]:
df_ta.date = pd.to_datetime(df_ta.date)
df_ta.groupby("team_api_id")["date"].min().describe()

count                    288
mean     2010-08-17 23:10:00
min      2010-02-22 00:00:00
25%      2010-02-22 00:00:00
50%      2010-02-22 00:00:00
75%      2010-02-22 00:00:00
max      2015-09-10 00:00:00
Name: date, dtype: object

In [2169]:
df_ta.date.value_counts()

date
2015-09-10    245
2014-09-19    244
2011-02-22    244
2012-02-22    242
2013-09-20    242
2010-02-22    241
Name: count, dtype: int64

In [2170]:
#we're dropping the matches from 2008 and 2009 for lack of stats, but that's okay, especially because
#there are fewer matches from 2008 in the dataset

    #List approach--.rename(columns = "...") would rename both and not distinguish match_date from (stats) date
cols = list(df_home.columns)
cols[cols.index("date")] = "match_date"
df_home.columns = cols

df_home.match_date = pd.to_datetime(df_home.match_date)
100 * df_home.match_date.dt.year.value_counts()/len(df_home)

match_date
2012    12.884519
2011    12.797956
2010    12.677613
2015    12.565010
2009    12.536860
2013    12.268022
2014    12.217351
2016     6.048152
2008     6.004518
Name: count, dtype: float64

#### Troubleshooting misalignment

In [2171]:
%%sql
home << SELECT home_team_api_id FROM Match;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable home


In [2172]:
home = home.DataFrame()

In [2173]:
%%sql
away << SELECT away_team_api_id FROM Match;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable away


In [2174]:
away = away.DataFrame()

In [2175]:
#lengths of home and away are the same
len(home), len(away)

(25979, 25979)

In [2176]:
#there are 299 unique teams
len(home.groupby("home_team_api_id")), len(away.groupby("away_team_api_id"))

(299, 299)

In [2177]:
#but only 288 teams have stats
len(df_ta.team_api_id.unique())

288

In [2178]:
%%sql
home_stats << WITH Home AS (SELECT home_team_api_id FROM Match)
SELECT * FROM
Home h INNER JOIN Team_Attributes ta ON h.home_team_api_id = ta.team_api_id;

 * sqlite:///../data/database.sqlite
Done.


Returning data to local variable home_stats


In [2179]:
home_stats = home_stats.DataFrame()

In [2180]:
%%sql
away_stats << WITH Away AS (SELECT away_team_api_id FROM Match)
SELECT * FROM
Away a INNER JOIN Team_Attributes ta ON a.away_team_api_id = ta.team_api_id;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable away_stats


In [2181]:
away_stats = away_stats.DataFrame()

In [2182]:
home_stats.shape, away_stats.shape

((142093, 26), (142100, 26))

LEFT JOIN: ((142271, 26), (142278, 26))

INNER JOIN: ((142093, 26), (142100, 26))

Seven rows difference either way--interesting

In [2183]:
#consistent with earlier finding that only 288 teams have stats
len(home_stats.groupby("home_team_api_id")), len(away_stats.groupby("away_team_api_id"))

(288, 288)

In [2184]:
#the id just counts up and is unique, so we can use it for our counts
df_ta.id

0          1
1          2
2          3
3          4
4          5
        ... 
1453    1454
1454    1455
1455    1456
1456    1457
1457    1458
Name: id, Length: 1458, dtype: int64

In [2185]:
#the number of stats measurements per team vary -- probably explaining the difference
df_ta.groupby("team_api_id")["id"].count().describe()

count    288.000000
mean       5.062500
std        1.540255
min        1.000000
25%        5.000000
50%        6.000000
75%        6.000000
max        6.000000
Name: id, dtype: float64

In [2186]:
#each team with stats is both a home team and an away team in the match table
unique_home = sorted([int(val) for val in set(home_stats["home_team_api_id"].values)])
unique_away = sorted([int(val) for val in set(away_stats["away_team_api_id"].values)])
unique_home == unique_away, len(unique_home)

(True, 288)

In [2187]:
#so every team is represented at least once, but different teams have different numbers
#of stats snapshots, hence the different numbers

In [2188]:
#need: a POC dataframe that goes match-by-match and has the nearest stats
#for each team before that match

## Pipeline

In [2189]:
%%sql
matches << WITH matches AS (SELECT match_api_id, country_id, league_id, season, stage, date, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal FROM Match)
SELECT * FROM matches m INNER JOIN Country c ON m.country_id = c.id INNER JOIN League l on m.league_id = l.id;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable matches


In [2190]:
matches = matches.DataFrame()

In [2191]:
cols = list(matches.columns)
cols[cols.index("date")] = "match_date"
matches.columns = cols
matches.match_date = pd.to_datetime(matches.match_date)
matches.head()

,match_api_id,country_id,league_id,season,stage,match_date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,id,name,id,country_id,name
0,492473,1,1,2008/2009,1,2008-08-17,9987,9993,1,1,1,Belgium,1,1,Belgium Jupiler League
1,492474,1,1,2008/2009,1,2008-08-16,10000,9994,0,0,1,Belgium,1,1,Belgium Jupiler League
2,492475,1,1,2008/2009,1,2008-08-16,9984,8635,0,3,1,Belgium,1,1,Belgium Jupiler League
3,492476,1,1,2008/2009,1,2008-08-17,9991,9998,5,0,1,Belgium,1,1,Belgium Jupiler League
4,492477,1,1,2008/2009,1,2008-08-16,7947,9985,1,3,1,Belgium,1,1,Belgium Jupiler League


In [2192]:
matches = matches.drop(columns = ["country_id", "league_id", "id"])
matches.head()

,match_api_id,season,stage,match_date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,name,name
0,492473,2008/2009,1,2008-08-17,9987,9993,1,1,Belgium,Belgium Jupiler League
1,492474,2008/2009,1,2008-08-16,10000,9994,0,0,Belgium,Belgium Jupiler League
2,492475,2008/2009,1,2008-08-16,9984,8635,0,3,Belgium,Belgium Jupiler League
3,492476,2008/2009,1,2008-08-17,9991,9998,5,0,Belgium,Belgium Jupiler League
4,492477,2008/2009,1,2008-08-16,7947,9985,1,3,Belgium,Belgium Jupiler League


In [2193]:
    #List approach--.rename(columns = "...") would rename both and not distinguish match_date from (stats) date
cols = list(matches.columns)
cols[cols.index("name")] = "Country"
matches.columns = cols
matches = matches.rename(columns = {"name":"League"})
matches.head()

,match_api_id,season,stage,match_date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,Country,League
0,492473,2008/2009,1,2008-08-17,9987,9993,1,1,Belgium,Belgium Jupiler League
1,492474,2008/2009,1,2008-08-16,10000,9994,0,0,Belgium,Belgium Jupiler League
2,492475,2008/2009,1,2008-08-16,9984,8635,0,3,Belgium,Belgium Jupiler League
3,492476,2008/2009,1,2008-08-17,9991,9998,5,0,Belgium,Belgium Jupiler League
4,492477,2008/2009,1,2008-08-16,7947,9985,1,3,Belgium,Belgium Jupiler League


In [2194]:
%%sql
ta_info << PRAGMA table_info(Team_Attributes)

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable ta_info


In [2195]:
ta_info = ta_info.DataFrame()

In [2196]:
ta_info.iloc[4:, :][ta_info.type == "INTEGER"]

/var/folders/p2/bl5l45jx3bxcyztvsjfffscm0000gn/T/ipykernel_54018/259395910.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ta_info.iloc[4:, :][ta_info.type == "INTEGER"]


,cid,name,type,notnull,dflt_value,pk
4,4,buildUpPlaySpeed,INTEGER,0,None,0
6,6,buildUpPlayDribbling,INTEGER,0,None,0
8,8,buildUpPlayPassing,INTEGER,0,None,0
11,11,chanceCreationPassing,INTEGER,0,None,0
13,13,chanceCreationCrossing,INTEGER,0,None,0
15,15,chanceCreationShooting,INTEGER,0,None,0
18,18,defencePressure,INTEGER,0,None,0
20,20,defenceAggression,INTEGER,0,None,0
22,22,defenceTeamWidth,INTEGER,0,None,0


In [2197]:
ta_info

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,team_fifa_api_id,INTEGER,0,None,0
2,2,team_api_id,INTEGER,0,None,0
3,3,date,TEXT,0,None,0
4,4,buildUpPlaySpeed,INTEGER,0,None,0
5,5,buildUpPlaySpeedClass,TEXT,0,None,0
6,6,buildUpPlayDribbling,INTEGER,0,None,0
7,7,buildUpPlayDribblingClass,TEXT,0,None,0
8,8,buildUpPlayPassing,INTEGER,0,None,0
9,9,buildUpPlayPassingClass,TEXT,0,None,0


In [2198]:
#get all the stats while eliminating the redundant classes; copy, paste, and
stats = list(ta_info.name)[4:]
stat_cols = [stat for stat in stats if stat[:-5] not in stats]
stat_cols
for stat in stat_cols:
    print(stat, end = ", ")

buildUpPlaySpeed, buildUpPlayDribbling, buildUpPlayPassing, buildUpPlayPositioningClass, chanceCreationPassing, chanceCreationCrossing, chanceCreationShooting, chanceCreationPositioningClass, defencePressure, defenceAggression, defenceTeamWidth, defenceDefenderLineClass, 

In [2199]:
%%sql
stats << SELECT team_api_id, date, buildUpPlaySpeed, buildUpPlayDribbling, buildUpPlayPassing, buildUpPlayPositioningClass, chanceCreationPassing, chanceCreationCrossing, chanceCreationShooting, chanceCreationPositioningClass, defencePressure, defenceAggression, defenceTeamWidth, defenceDefenderLineClass FROM Team_Attributes;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable stats


In [2200]:
stats = stats.DataFrame()

In [2201]:
stats.date = pd.to_datetime(stats.date)
stats.head()

,team_api_id,date,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,chanceCreationPositioningClass,defencePressure,defenceAggression,defenceTeamWidth,defenceDefenderLineClass
0,9930,2010-02-22,60,NaN,50,Organised,60,65,55,Organised,50,55,45,Cover
1,9930,2014-09-19,52,48.0,56,Organised,54,63,64,Organised,47,44,54,Cover
2,9930,2015-09-10,47,41.0,54,Organised,54,63,64,Organised,47,44,54,Cover
3,8485,2010-02-22,70,NaN,70,Organised,70,70,70,Organised,60,70,70,Cover
4,8485,2011-02-22,47,NaN,52,Organised,53,48,52,Organised,47,47,52,Cover


In [2202]:
matches = matches.sort_values(by = "match_date")
matches.head()

,match_api_id,season,stage,match_date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,Country,League
24558,486263,2008/2009,1,2008-07-18,10192,9931,1,2,Switzerland,Switzerland Super League
24559,486264,2008/2009,1,2008-07-19,9930,10179,3,1,Switzerland,Switzerland Super League
24560,486265,2008/2009,1,2008-07-20,10199,9824,1,2,Switzerland,Switzerland Super League
24561,486266,2008/2009,1,2008-07-20,7955,10243,1,2,Switzerland,Switzerland Super League
24613,486268,2008/2009,2,2008-07-23,6493,7955,1,2,Switzerland,Switzerland Super League


In [2203]:
stats = stats.sort_values(by = "date")
stats = stats.rename(columns = {"team_api_id":"home_team_api_id"})
stats.head()

,home_team_api_id,date,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,chanceCreationPositioningClass,defencePressure,defenceAggression,defenceTeamWidth,defenceDefenderLineClass
0,9930,2010-02-22,60,NaN,50,Organised,60,65,55,Organised,50,55,45,Cover
426,8674,2010-02-22,41,NaN,32,Organised,40,47,69,Organised,30,30,30,Cover
1147,10189,2010-02-22,65,NaN,55,Organised,55,70,70,Organised,70,45,70,Cover
419,8722,2010-02-22,55,NaN,65,Organised,65,40,60,Organised,45,55,70,Cover
418,8596,2010-02-22,70,NaN,70,Organised,60,70,70,Organised,60,70,70,Cover


In [2204]:
#for each match, add the most recent home team stats before the date of that match
df_home = pd.merge_asof(matches, stats, left_on = "match_date", right_on = "date", by = "home_team_api_id", direction = "backward")
df_home[df_home["defencePressure"] > 0].head(20)

,match_api_id,season,stage,match_date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,Country,League,...,buildUpPlayPassing,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,chanceCreationPositioningClass,defencePressure,defenceAggression,defenceTeamWidth,defenceDefenderLineClass
5469,684955,2009/2010,23,2010-02-22,10267,8305,2,1,Spain,Spain LIGA BBVA,...,30.0,Free Form,55.0,60.0,70.0,Organised,55.0,60.0,60.0,Offside Trap
5470,686171,2009/2010,20,2010-02-22,9807,10215,1,2,Portugal,Portugal Liga ZON Sagres,...,30.0,Organised,55.0,45.0,60.0,Organised,30.0,30.0,30.0,Offside Trap
5471,659091,2009/2010,28,2010-02-23,10260,8654,3,0,England,England Premier League,...,45.0,Organised,45.0,70.0,65.0,Free Form,40.0,50.0,40.0,Cover
5472,704654,2009/2010,17,2010-02-24,8600,8529,2,1,Italy,Italy Serie A,...,35.0,Organised,30.0,50.0,70.0,Organised,35.0,30.0,55.0,Offside Trap
5473,704632,2009/2010,17,2010-02-24,8535,8564,1,2,Italy,Italy Serie A,...,30.0,Free Form,50.0,60.0,60.0,Free Form,45.0,30.0,60.0,Offside Trap
5474,665677,2009/2010,23,2010-02-24,9984,9986,1,0,Belgium,Belgium Jupiler League,...,35.0,Organised,70.0,45.0,55.0,Organised,65.0,60.0,70.0,Cover
5475,665694,2009/2010,24,2010-02-24,9999,9987,1,1,Belgium,Belgium Jupiler League,...,60.0,Organised,50.0,35.0,40.0,Organised,65.0,70.0,70.0,Cover
5476,674600,2009/2010,18,2010-02-26,2186,8673,1,2,Poland,Poland Ekstraklasa,...,45.0,Organised,40.0,35.0,65.0,Organised,70.0,70.0,65.0,Cover
5477,686186,2009/2010,21,2010-02-26,9771,7844,0,1,Portugal,Portugal Liga ZON Sagres,...,50.0,Organised,50.0,50.0,50.0,Organised,50.0,50.0,50.0,Cover
5478,674483,2009/2010,24,2010-02-26,10189,9789,2,1,Germany,Germany 1. Bundesliga,...,55.0,Organised,55.0,70.0,70.0,Organised,70.0,45.0,70.0,Cover


In [2205]:
dates_check = df_home.groupby(["home_team_api_id", "date"])[["match_date"]].min()
dates_check.head(20)

match_date
home_team_api_id date                 
1601             2010-02-22 2010-03-06
                 2011-02-22 2011-02-26
                 2012-02-22 2012-02-24
                 2013-09-20 2013-09-22
                 2014-09-19 2014-09-21
                 2015-09-10 2015-09-13
1773             2012-02-22 2012-03-03
                 2014-09-19 2015-08-02
1957             2010-02-22 2010-03-12
                 2011-02-22 2011-03-08
                 2012-02-22 2012-03-04
                 2013-09-20 2013-09-25
                 2014-09-19 2014-09-29
                 2015-09-10 2015-09-20
2033             2010-02-22 2010-03-14
                 2011-02-22 2011-02-26
                 2012-02-22 2012-02-26
                 2013-09-20 2013-09-28
2182             2010-02-22 2010-03-06
                 2011-02-22 2011-02-27

In [2206]:
#is every match date on or after the date of the stats snapshot?
stat_dates = [j for i, j in dates_check.index]
count = 0
for i in range(len(dates_check)):
    if list(dates_check.values)[i] >= stat_dates[i]: #that is, if the match date is equal to or greater than the date on which stats were last measured
        count += 1
print(f"{count == len(dates_check)}: {count, len(dates_check)}")

True: (1171, 1171)


In [2207]:
stats = stats.rename(columns = {"home_team_api_id":"away_team_api_id"})
matches.columns, stats.columns

(Index(['match_api_id', 'season', 'stage', 'match_date', 'home_team_api_id',
        'away_team_api_id', 'home_team_goal', 'away_team_goal', 'Country',
        'League'],
       dtype='object'),
 Index(['away_team_api_id', 'date', 'buildUpPlaySpeed', 'buildUpPlayDribbling',
        'buildUpPlayPassing', 'buildUpPlayPositioningClass',
        'chanceCreationPassing', 'chanceCreationCrossing',
        'chanceCreationShooting', 'chanceCreationPositioningClass',
        'defencePressure', 'defenceAggression', 'defenceTeamWidth',
        'defenceDefenderLineClass'],
       dtype='object'))

In [2208]:
df_away = pd.merge_asof(matches, stats, left_on = "match_date", right_on = "date", by = "away_team_api_id", direction = "backward")
df_away[df_away["defencePressure"] > 0].head()
df_away_stats = df_away.iloc[:, -12:]
df_away_stats

,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,chanceCreationPositioningClass,defencePressure,defenceAggression,defenceTeamWidth,defenceDefenderLineClass
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
25974,50.0,60.0,54.0,Organised,52.0,54.0,46.0,Organised,36.0,43.0,47.0,Cover
25975,52.0,46.0,48.0,Organised,39.0,39.0,37.0,Organised,40.0,36.0,52.0,Cover
25976,61.0,57.0,37.0,Organised,62.0,50.0,53.0,Organised,45.0,47.0,54.0,Cover
25977,52.0,56.0,64.0,Organised,39.0,66.0,46.0,Organised,44.0,34.0,50.0,Cover


In [2209]:
#df_home.iloc[:, -2:] = df_home.iloc[:, -2:].add_prefix("home_")
df_away_stats = df_away_stats.add_prefix("away_")

both_stats = pd.concat([df_home, df_away_stats], axis = 1)

In [2210]:
both_stats = both_stats.dropna(subset = list(both_stats.columns)[-4:])

In [2211]:
both_stats.head()

,match_api_id,season,stage,match_date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,Country,League,...,away_buildUpPlayPassing,away_buildUpPlayPositioningClass,away_chanceCreationPassing,away_chanceCreationCrossing,away_chanceCreationShooting,away_chanceCreationPositioningClass,away_defencePressure,away_defenceAggression,away_defenceTeamWidth,away_defenceDefenderLineClass
5469,684955,2009/2010,23,2010-02-22,10267,8305,2,1,Spain,Spain LIGA BBVA,...,35.0,Organised,35.0,50.0,70.0,Free Form,40.0,30.0,50.0,Offside Trap
5470,686171,2009/2010,20,2010-02-22,9807,10215,1,2,Portugal,Portugal Liga ZON Sagres,...,30.0,Organised,50.0,60.0,55.0,Organised,30.0,30.0,30.0,Offside Trap
5471,659091,2009/2010,28,2010-02-23,10260,8654,3,0,England,England Premier League,...,30.0,Organised,31.0,70.0,50.0,Organised,30.0,70.0,30.0,Cover
5472,704654,2009/2010,17,2010-02-24,8600,8529,2,1,Italy,Italy Serie A,...,30.0,Organised,30.0,35.0,55.0,Organised,40.0,55.0,60.0,Cover
5473,704632,2009/2010,17,2010-02-24,8535,8564,1,2,Italy,Italy Serie A,...,30.0,Free Form,55.0,45.0,70.0,Free Form,30.0,35.0,60.0,Offside Trap


In [2212]:
both_stats_cols = list(both_stats.columns)
both_stats_cols

['match_api_id',
 'season',
 'stage',
 'match_date',
 'home_team_api_id',
 'away_team_api_id',
 'home_team_goal',
 'away_team_goal',
 'Country',
 'League',
 'date',
 'buildUpPlaySpeed',
 'buildUpPlayDribbling',
 'buildUpPlayPassing',
 'buildUpPlayPositioningClass',
 'chanceCreationPassing',
 'chanceCreationCrossing',
 'chanceCreationShooting',
 'chanceCreationPositioningClass',
 'defencePressure',
 'defenceAggression',
 'defenceTeamWidth',
 'defenceDefenderLineClass',
 'away_buildUpPlaySpeed',
 'away_buildUpPlayDribbling',
 'away_buildUpPlayPassing',
 'away_buildUpPlayPositioningClass',
 'away_chanceCreationPassing',
 'away_chanceCreationCrossing',
 'away_chanceCreationShooting',
 'away_chanceCreationPositioningClass',
 'away_defencePressure',
 'away_defenceAggression',
 'away_defenceTeamWidth',
 'away_defenceDefenderLineClass']

In [2213]:
start_idx = both_stats_cols.index("buildUpPlaySpeed")
raw_stats_cols = [stat for stat in both_stats_cols[start_idx:] if not stat.startswith("away_")]

In [2214]:
both_stats_cols = list(both_stats.columns)
start_idx = both_stats_cols.index("buildUpPlaySpeed")
raw_stats_cols = [stat for stat in both_stats_cols[start_idx:] if not stat.startswith("away_")]


for stat in raw_stats_cols: 
    if ("int" in str(both_stats[f"{stat}"].dtype)) | ("float" in str(both_stats[f"{stat}"].dtype)):
        both_stats[f"{stat}_home_diff"] = both_stats[f"{stat}"] - both_stats[f"away_{stat}"]
        both_stats = both_stats.drop(columns = [f"{stat}", f"away_{stat}"])

In [2215]:
both_stats = both_stats.drop(columns = ["home_team_api_id", "away_team_api_id", "date"])
both_stats.head()

,match_api_id,season,stage,match_date,home_team_goal,away_team_goal,Country,League,buildUpPlayPositioningClass,chanceCreationPositioningClass,...,away_defenceDefenderLineClass,buildUpPlaySpeed_home_diff,buildUpPlayDribbling_home_diff,buildUpPlayPassing_home_diff,chanceCreationPassing_home_diff,chanceCreationCrossing_home_diff,chanceCreationShooting_home_diff,defencePressure_home_diff,defenceAggression_home_diff,defenceTeamWidth_home_diff
5469,684955,2009/2010,23,2010-02-22,2,1,Spain,Spain LIGA BBVA,Free Form,Organised,...,Offside Trap,0.0,NaN,-5.0,20.0,10.0,0.0,15.0,30.0,10.0
5470,686171,2009/2010,20,2010-02-22,1,2,Portugal,Portugal Liga ZON Sagres,Organised,Organised,...,Offside Trap,0.0,NaN,0.0,5.0,-15.0,5.0,0.0,0.0,0.0
5471,659091,2009/2010,28,2010-02-23,3,0,England,England Premier League,Organised,Free Form,...,Cover,12.0,NaN,15.0,14.0,0.0,15.0,10.0,-20.0,10.0
5472,704654,2009/2010,17,2010-02-24,2,1,Italy,Italy Serie A,Organised,Organised,...,Cover,24.0,NaN,5.0,0.0,15.0,15.0,-5.0,-25.0,-5.0
5473,704632,2009/2010,17,2010-02-24,1,2,Italy,Italy Serie A,Free Form,Free Form,...,Offside Trap,22.0,NaN,0.0,-5.0,15.0,-10.0,15.0,-5.0,0.0


In [2216]:
#def win_lose_draw(row):
#    if row["home_team_goal"] > row["away_team_goal"]:
#        return "W"
#    if row["home_team_goal"] < row["away_team_goal"]:
#        return "L"
#    else:
#        return "D"

#both_stats["Outcome"] = both_stats.apply(win_lose_draw, axis = 1)

In [2217]:
both_stats["Home_Score_Adv"] = both_stats["home_team_goal"] - both_stats["away_team_goal"]
both_stats = both_stats.drop(columns = ["home_team_goal", "away_team_goal"])
both_stats = both_stats.reset_index().drop(columns = "index")

In [2218]:
both_stats.head(20)
#it's all mixed up because it was ordered by match date; that's okay (i think)
#should season be there? and does the concept of stage apply no matter the exact time period (e.g. 2010s vs 2020s?)

,match_api_id,season,stage,match_date,Country,League,buildUpPlayPositioningClass,chanceCreationPositioningClass,defenceDefenderLineClass,away_buildUpPlayPositioningClass,...,buildUpPlaySpeed_home_diff,buildUpPlayDribbling_home_diff,buildUpPlayPassing_home_diff,chanceCreationPassing_home_diff,chanceCreationCrossing_home_diff,chanceCreationShooting_home_diff,defencePressure_home_diff,defenceAggression_home_diff,defenceTeamWidth_home_diff,Home_Score_Adv
0,684955,2009/2010,23,2010-02-22,Spain,Spain LIGA BBVA,Free Form,Organised,Offside Trap,Organised,...,0.0,NaN,-5.0,20.0,10.0,0.0,15.0,30.0,10.0,1
1,686171,2009/2010,20,2010-02-22,Portugal,Portugal Liga ZON Sagres,Organised,Organised,Offside Trap,Organised,...,0.0,NaN,0.0,5.0,-15.0,5.0,0.0,0.0,0.0,-1
2,659091,2009/2010,28,2010-02-23,England,England Premier League,Organised,Free Form,Cover,Organised,...,12.0,NaN,15.0,14.0,0.0,15.0,10.0,-20.0,10.0,3
3,704654,2009/2010,17,2010-02-24,Italy,Italy Serie A,Organised,Organised,Offside Trap,Organised,...,24.0,NaN,5.0,0.0,15.0,15.0,-5.0,-25.0,-5.0,1
4,704632,2009/2010,17,2010-02-24,Italy,Italy Serie A,Free Form,Free Form,Offside Trap,Free Form,...,22.0,NaN,0.0,-5.0,15.0,-10.0,15.0,-5.0,0.0,-1
5,665677,2009/2010,23,2010-02-24,Belgium,Belgium Jupiler League,Organised,Organised,Cover,Organised,...,5.0,NaN,-15.0,25.0,2.0,-5.0,-5.0,-10.0,0.0,1
6,665694,2009/2010,24,2010-02-24,Belgium,Belgium Jupiler League,Organised,Organised,Cover,Organised,...,20.0,NaN,15.0,0.0,0.0,-20.0,-5.0,5.0,0.0,0
7,674600,2009/2010,18,2010-02-26,Poland,Poland Ekstraklasa,Organised,Organised,Cover,Organised,...,0.0,NaN,10.0,-15.0,-10.0,-5.0,5.0,0.0,5.0,-1
8,686186,2009/2010,21,2010-02-26,Portugal,Portugal Liga ZON Sagres,Organised,Organised,Cover,Organised,...,20.0,NaN,20.0,-5.0,-5.0,-5.0,20.0,20.0,20.0,-1
9,674483,2009/2010,24,2010-02-26,Germany,Germany 1. Bundesliga,Organised,Organised,Cover,Organised,...,-5.0,NaN,10.0,-15.0,36.0,4.0,16.0,-25.0,17.0,1


# Joining Player Attributes

In [2219]:
#structure:
    #match -- all the home player api ids and all the away player api ids
    #player_attributes -- player api id, date, and player stats
    #

#big idea:
#for each match, we want columns that represent the differences in the averages of each player stat across the teams
    #calculate averages of each player stat

In [2220]:
%%sql
SELECT * FROM Match
ORDER BY date
LIMIT 20;

 * sqlite:///../data/database.sqlite
Done.


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
24559,24558,24558,2008/2009,1,2008-07-18 00:00:00,486263,10192,9931,1,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
24560,24558,24558,2008/2009,1,2008-07-19 00:00:00,486264,9930,10179,3,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
24561,24558,24558,2008/2009,1,2008-07-20 00:00:00,486265,10199,9824,1,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
24562,24558,24558,2008/2009,1,2008-07-20 00:00:00,486266,7955,10243,1,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
24613,24558,24558,2008/2009,2,2008-07-23 00:00:00,486267,9931,9956,1,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,N

In [2221]:
%%sql
SELECT * FROM Player_Attributes
ORDER BY player_api_id, date
LIMIT 20;

 * sqlite:///../data/database.sqlite
Done.


id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
139857,148544,2625,2007-02-22 00:00:00,63,64,right,high,medium,48,48,47,64,38,57,50,46,67,57,67,64,59,52,49,61,56,78,56,59,72,52,55,56,46,64,66,63,14,11,67,9,10
139856,148544,2625,2007-08-30 00:00:00,63,64,right,high,medium,48,48,47,64,38,57,50,51,67,57,67,64,59,52,49,61,56,78,56,59,72,52,55,56,46,64,66,63,14,24,67,24,24
139855,148544,2625,2008-08-30 00:00:00,60,64,right,high,medium,48,48,47,64,38,57,50,51,67,57,67,64,59,52,49,61,56,78,56,59,72,52,55,56,46,64,66,63,14,24,67,24,24
139854,148544,2625,2010-08-30 00:00:00,60,64,right,high,medium,48,48,47,64,38,57,50,51,67,57,67,64,59,50,49,71,56,78,56,59,72,71,50,56,69,64,66,63,12,11,6,8,8
139853,148544,2625,2011-02-22 00:00:00,59,63,right,high,medium,52,47,46,63,37,56,49,50,66,58,66,63,58,49,48,68,55,77,55,58,71,70,49,55,66,63,63,62,12,11,6,8,8
139852,148544,2625,2011-08-30 00:00:00,58,58,right,high,medium,52,47,46,53,37,56,49,50,66,58,66,63,58,49,75,68,55,77,55,58,71,60,49,55,66,63,63,52,12,11,6,8,8
139851,148544,2625,2012-02-22 00:00:00,58,58,right,medium,medium,52,47,46,53,37,56,49,50,66,58,67,63,60,49,76,68,55,77,55,58,71,64,49,55,66,63,63,52,12,11,6,8,8
139850,148544,2625,2012-08-31 00:00:00,58,58,right,medium,medium,52,47,46,53,37,56,49,50,66,58,67,63,74,49,76,68,55,77,56,58,71,64,49,55,66,63,63,52,12,11,6,8,8
139849,148544,2625,2013-02-15 00:00:00,58,58,right,medium,medium,52,47,46,53,37,56,49,50,66,58,67,63,74,49,76,68,63,77,56,58,71,64,49,55,66,63,63,52,12,11,6,8,8
139848,148544,2625,2013-03-08 00:00:00,61,61,right,medium,medium,50,47,46,52,39,50,51,50,64,62,67,63,74,49,76,68,63,77,56,54,71,64,49,55,66,62,63,54,12,11,6,8,8


In [2222]:
%%sql
SELECT * FROM Match
ORDER BY date
LIMIT 10;

 * sqlite:///../data/database.sqlite
Done.


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
24559,24558,24558,2008/2009,1,2008-07-18 00:00:00,486263,10192,9931,1,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
24560,24558,24558,2008/2009,1,2008-07-19 00:00:00,486264,9930,10179,3,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
24561,24558,24558,2008/2009,1,2008-07-20 00:00:00,486265,10199,9824,1,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
24562,24558,24558,2008/2009,1,2008-07-20 00:00:00,486266,7955,10243,1,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
24613,24558,24558,2008/2009,2,2008-07-23 00:00:00,486267,9931,9956,1,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,N

In [2223]:
%%sql
pa << SELECT * FROM Player_Attributes;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable pa


In [2224]:
pa = pa.DataFrame()
pa.date = pd.to_datetime(pa.date)
pa.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [2225]:
core_attributes = list(pa.columns)[4:-5]
for object in ["preferred_foot", "attacking_work_rate", "defensive_work_rate"]:
    core_attributes.remove(object)
query_attributes = ", ".join(core_attributes)
query_attributes
alias_main_attributes = []

In [2226]:
query_attributes

'overall_rating, potential, crossing, finishing, heading_accuracy, short_passing, volleys, dribbling, curve, free_kick_accuracy, long_passing, ball_control, acceleration, sprint_speed, agility, reactions, balance, shot_power, jumping, stamina, strength, long_shots, aggression, interceptions, positioning, vision, penalties, marking, standing_tackle, sliding_tackle'

In [2227]:
for team in ["home", "away"]:
    for num in range(1,12):
        for att in core_attributes:
            alias_main_attributes.append(f"{att} AS {team}_{num}_{att}")
print(alias_main_attributes)
       
query_alias_main_attributes = ", ".join(alias_main_attributes)
query_alias_main_attributes

['overall_rating AS home_1_overall_rating', 'potential AS home_1_potential', 'crossing AS home_1_crossing', 'finishing AS home_1_finishing', 'heading_accuracy AS home_1_heading_accuracy', 'short_passing AS home_1_short_passing', 'volleys AS home_1_volleys', 'dribbling AS home_1_dribbling', 'curve AS home_1_curve', 'free_kick_accuracy AS home_1_free_kick_accuracy', 'long_passing AS home_1_long_passing', 'ball_control AS home_1_ball_control', 'acceleration AS home_1_acceleration', 'sprint_speed AS home_1_sprint_speed', 'agility AS home_1_agility', 'reactions AS home_1_reactions', 'balance AS home_1_balance', 'shot_power AS home_1_shot_power', 'jumping AS home_1_jumping', 'stamina AS home_1_stamina', 'strength AS home_1_strength', 'long_shots AS home_1_long_shots', 'aggression AS home_1_aggression', 'interceptions AS home_1_interceptions', 'positioning AS home_1_positioning', 'vision AS home_1_vision', 'penalties AS home_1_penalties', 'marking AS home_1_marking', 'standing_tackle AS home_

'overall_rating AS home_1_overall_rating, potential AS home_1_potential, crossing AS home_1_crossing, finishing AS home_1_finishing, heading_accuracy AS home_1_heading_accuracy, short_passing AS home_1_short_passing, volleys AS home_1_volleys, dribbling AS home_1_dribbling, curve AS home_1_curve, free_kick_accuracy AS home_1_free_kick_accuracy, long_passing AS home_1_long_passing, ball_control AS home_1_ball_control, acceleration AS home_1_acceleration, sprint_speed AS home_1_sprint_speed, agility AS home_1_agility, reactions AS home_1_reactions, balance AS home_1_balance, shot_power AS home_1_shot_power, jumping AS home_1_jumping, stamina AS home_1_stamina, strength AS home_1_strength, long_shots AS home_1_long_shots, aggression AS home_1_aggression, interceptions AS home_1_interceptions, positioning AS home_1_positioning, vision AS home_1_vision, penalties AS home_1_penalties, marking AS home_1_marking, standing_tackle AS home_1_standing_tackle, sliding_tackle AS home_1_sliding_tackl

In [2228]:
#Encoding the categorical player attributes somehow--turning that into some kind of aggregated
#value (e.g., most common) fo each team and even trying to derive some sort of relationship between teams--
#doesn't seem useful for the model or worth the time and computation.

In [2229]:
start_idx, stop_idx = 0, 30

In [2230]:
db = DATA_PATH.joinpath("database.sqlite")
con = sqlite3.connect(db)
dfs = []

core_attributes = list(pa.columns)[4:-5] #all the player stats minus the GK stats
for object in ["preferred_foot", "attacking_work_rate", "defensive_work_rate"]:
    core_attributes.remove(object)

alias_main_attributes = []

for team in ["home", "away"]:
    for num in range(1,12):
        for att in core_attributes:
            alias_main_attributes.append(f"{att} AS {team}_{num}_{att}")   
        caa_slice = alias_main_attributes[start_idx:stop_idx]
        query_alias_main_attributes = ", ".join(caa_slice)
        start_idx += 30
        stop_idx += 30
        print(query_alias_main_attributes)

overall_rating AS home_1_overall_rating, potential AS home_1_potential, crossing AS home_1_crossing, finishing AS home_1_finishing, heading_accuracy AS home_1_heading_accuracy, short_passing AS home_1_short_passing, volleys AS home_1_volleys, dribbling AS home_1_dribbling, curve AS home_1_curve, free_kick_accuracy AS home_1_free_kick_accuracy, long_passing AS home_1_long_passing, ball_control AS home_1_ball_control, acceleration AS home_1_acceleration, sprint_speed AS home_1_sprint_speed, agility AS home_1_agility, reactions AS home_1_reactions, balance AS home_1_balance, shot_power AS home_1_shot_power, jumping AS home_1_jumping, stamina AS home_1_stamina, strength AS home_1_strength, long_shots AS home_1_long_shots, aggression AS home_1_aggression, interceptions AS home_1_interceptions, positioning AS home_1_positioning, vision AS home_1_vision, penalties AS home_1_penalties, marking AS home_1_marking, standing_tackle AS home_1_standing_tackle, sliding_tackle AS home_1_sliding_tackle

In [2231]:
%%sql
matches << SELECT * FROM Match;

 * sqlite:///../data/database.sqlite
Done.
Returning data to local variable matches


In [2232]:
matches = pd.DataFrame(matches)
xy = matches.dropna()[list(matches.columns)[11:55]]
xy_agg = xy[list(xy.columns)[22:44]].agg(("min", "max"))
xy_agg.iloc[:, :11]

,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11
min,1.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0,7.0,6.0,10.0
max,1.0,3.0,3.0,3.0,7.0,8.0,8.0,9.0,10.0,11.0,11.0


In [2233]:
xy_agg.iloc[:, 11:]

,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11
min,1.0,3.0,3.0,3.0,3.0,3.0,5.0,5.0,6.0,6.0,10.0
max,1.0,3.0,3.0,3.0,7.0,7.0,8.0,9.0,10.0,11.0,11.0


In [2234]:
len(xy.home_player_Y1.unique()) == len(xy.away_player_Y1.unique()) == 1
#confirms that player 1 is always the goal keeper

True

In [2235]:
#running these outside the main loop saves on computing
gk_attributes = list(pa.columns)[4:] #all the player stats, now including GK stats
for object in ["preferred_foot", "attacking_work_rate", "defensive_work_rate"]:
    gk_attributes.remove(object)

home_alias_gk_attributes = []
away_alias_gk_attributes = []

for team in ["home", "away"]:
    for att in gk_attributes:
        if team == "home":
            home_alias_gk_attributes.append(f"{att} AS home_1_{att}")
        else:
            away_alias_gk_attributes.append(f"{att} AS away_1_{att}")
            
query_home_alias_gk_attributes = ", ".join(home_alias_gk_attributes)
query_away_alias_gk_attributes = ", ".join(away_alias_gk_attributes) 

In [2236]:
start_idx, stop_idx = 0, 30

for team in ["home", "away"]:
    for num in range(1, 12):
        query_1 = f"SELECT match_api_id, date AS match_date, {team}_player_{num} FROM Match;"

        df1 = pd.read_sql(sql = query_1, con = con)
        df1.match_date = pd.to_datetime(df1.match_date)
        df1 = df1.sort_values(by = "match_date")
        df1[f"{team}_player_{num}"] = df1[f"{team}_player_{num}"].astype(float) #required to prevent error

        caa_slice = alias_main_attributes[start_idx:stop_idx]
        query_alias_main_attributes = ", ".join(caa_slice)
        start_idx += 30
        stop_idx += 30

        if num == 1:
            if team == "home":
                query_2 = f"SELECT player_api_id, date AS {team}_{num}_stat_date, {query_home_alias_gk_attributes} FROM Player_Attributes;"
            else:
                query_2 = f"SELECT player_api_id, date AS {team}_{num}_stat_date, {query_away_alias_gk_attributes} FROM Player_Attributes;"
        else:
            query_2 = f"SELECT player_api_id, date AS {team}_{num}_stat_date, {query_alias_main_attributes} FROM Player_Attributes;"

            
        df2 = pd.read_sql(sql = query_2, con = con)
        df2.player_api_id = df2.player_api_id.astype(float) #required to prevent error
        df2[f"{team}_{num}_stat_date"] = pd.to_datetime(df2[f"{team}_{num}_stat_date"])
        df2 = df2.sort_values(by = f"{team}_{num}_stat_date")
        
        df3 = pd.merge_asof(df1, df2, left_by = f"{team}_player_{num}", right_by = "player_api_id", left_on = "match_date", right_on = f"{team}_{num}_stat_date", direction = "backward")
        df3 = df3.drop(columns = ["player_api_id", f"{team}_player_{num}"])
        if not (num == 1) & (team == "home"):
            df3 = df3.drop(columns = ["match_date", "match_api_id"]) #match date required for use in merge_asof lookup but no longer needed in DataFrame after first iteration
        dfs.append(df3)

In [2237]:
df = pd.concat(dfs, axis = 1)
df = df.dropna()

stat_dates = [f"{team}_{num}_stat_date" for team in ["home", "away"] for num in range(1,12)]
stat_dates.insert(0, "match_date")
df_dates = df[stat_dates]
df_dates

#This verifies that, for each match, all the players' stats measurements were made before the match date
sum(df_dates.iloc[:, 1:].max(axis = 1) <= df_dates.match_date) == len(df_dates)

True

In [2238]:
df = df.drop(columns = stat_dates)

In [2239]:
df.head()

,match_api_id,home_1_overall_rating,home_1_potential,home_1_crossing,home_1_finishing,home_1_heading_accuracy,home_1_short_passing,home_1_volleys,home_1_dribbling,home_1_curve,...,away_11_strength,away_11_long_shots,away_11_aggression,away_11_interceptions,away_11_positioning,away_11_vision,away_11_penalties,away_11_marking,away_11_standing_tackle,away_11_sliding_tackle
28,483134,69.0,75.0,61.0,30.0,62.0,72.0,32.0,44.0,42.0,...,71.0,60.0,55.0,33.0,65.0,57.0,71.0,15.0,21.0,11.0
43,483133,75.0,82.0,22.0,22.0,23.0,22.0,21.0,22.0,11.0,...,70.0,67.0,55.0,59.0,78.0,78.0,52.0,22.0,25.0,12.0
52,489050,68.0,79.0,21.0,20.0,26.0,27.0,21.0,30.0,8.0,...,78.0,75.0,59.0,70.0,76.0,72.0,72.0,37.0,50.0,28.0
77,489044,78.0,85.0,22.0,21.0,21.0,21.0,9.0,21.0,9.0,...,76.0,75.0,63.0,79.0,89.0,81.0,86.0,21.0,24.0,29.0
78,489043,75.0,78.0,22.0,22.0,22.0,22.0,10.0,22.0,11.0,...,80.0,45.0,66.0,59.0,63.0,54.0,57.0,22.0,31.0,20.0


In [2240]:
#This verifies that only the 1 players have the five GK attributes
gk_atts = [q for q in list(df.columns) if "gk" in str(q)]
gk_atts

['home_1_gk_diving',
 'home_1_gk_handling',
 'home_1_gk_kicking',
 'home_1_gk_positioning',
 'home_1_gk_reflexes',
 'away_1_gk_diving',
 'away_1_gk_handling',
 'away_1_gk_kicking',
 'away_1_gk_positioning',
 'away_1_gk_reflexes']

In [2241]:
df = df.reset_index().drop(columns = "index")

In [2242]:
home_rating_test_cols = [str(col).strip() for col in list(df.columns) if "home" in col and "overall_rating" in col]
print(home_rating_test_cols)

#now I can change the index and make sure this works. It does!
idx = 4

val_list = []
for val in df.loc[idx, home_rating_test_cols]:
    val_list.append(val)
    
print(val_list)

print(df.loc[idx, home_rating_test_cols].mean(), sum(val_list)/len(val_list))

df.loc[idx, home_rating_test_cols]

['home_1_overall_rating', 'home_2_overall_rating', 'home_3_overall_rating', 'home_4_overall_rating', 'home_5_overall_rating', 'home_6_overall_rating', 'home_7_overall_rating', 'home_8_overall_rating', 'home_9_overall_rating', 'home_10_overall_rating', 'home_11_overall_rating']
[75.0, 76.0, 85.0, 74.0, 77.0, 72.0, 79.0, 75.0, 85.0, 87.0, 73.0]
78.0 78.0


home_1_overall_rating     75.0
home_2_overall_rating     76.0
home_3_overall_rating     85.0
home_4_overall_rating     74.0
home_5_overall_rating     77.0
home_6_overall_rating     72.0
home_7_overall_rating     79.0
home_8_overall_rating     75.0
home_9_overall_rating     85.0
home_10_overall_rating    87.0
home_11_overall_rating    73.0
Name: 4, dtype: float64

In [2243]:
avg_player_stats = pd.DataFrame(df.match_api_id)

In [2244]:
for team in ["home", "away"]:
    for att in core_attributes:
        cols = [str(col).strip() for col in list(df.columns) if (f"{team}" in col) & (f"{att}" in col)]
        avg_player_stats[f"avg_{team}_player_{att}"] = df[cols].mean(axis = 1)

In [2245]:
avg_player_stats[["avg_home_player_overall_rating", "avg_away_player_overall_rating"]]
#yep, looks like the differences were calculated correctly

,avg_home_player_overall_rating,avg_away_player_overall_rating
0,67.909091,73.363636
1,78.818182,70.363636
2,68.272727,74.909091
3,73.727273,79.909091
4,78.000000,71.181818
...,...,...
18846,68.000000,64.545455
18847,59.363636,63.181818
18848,65.727273,67.636364
18849,60.181818,68.363636


In [2246]:
avg_player_stats.head()

,match_api_id,avg_home_player_overall_rating,avg_home_player_potential,avg_home_player_crossing,avg_home_player_finishing,avg_home_player_heading_accuracy,avg_home_player_short_passing,avg_home_player_volleys,avg_home_player_dribbling,avg_home_player_curve,...,avg_away_player_strength,avg_away_player_long_shots,avg_away_player_aggression,avg_away_player_interceptions,avg_away_player_positioning,avg_away_player_vision,avg_away_player_penalties,avg_away_player_marking,avg_away_player_standing_tackle,avg_away_player_sliding_tackle
0,483134,67.909091,75.454545,51.454545,44.545455,60.727273,61.545455,42.727273,49.636364,46.454545,...,69.000000,50.818182,67.181818,68.363636,68.166667,70.363636,70.818182,54.363636,53.909091,54.090909
1,483133,78.818182,85.272727,67.818182,59.000000,70.363636,73.363636,58.909091,64.818182,63.181818,...,69.363636,53.272727,64.181818,67.272727,66.083333,70.545455,58.636364,53.272727,54.909091,52.909091
2,489050,68.272727,74.363636,56.454545,50.636364,59.727273,58.454545,49.090909,54.636364,46.000000,...,70.181818,60.545455,69.454545,75.818182,74.833333,69.181818,74.363636,53.545455,57.000000,54.454545
3,489044,73.727273,78.727273,63.000000,53.909091,57.818182,62.272727,52.090909,58.545455,58.545455,...,74.545455,55.090909,70.818182,77.454545,77.333333,73.818182,76.000000,54.454545,58.090909,54.545455
4,489043,78.000000,84.545455,60.545455,51.454545,64.909091,65.454545,59.181818,66.727273,60.090909,...,66.818182,51.636364,69.000000,65.272727,64.750000,63.909091,67.636364,53.272727,57.181818,54.545455


In [2247]:
home_diffs = pd.DataFrame(df.match_api_id)

In [2248]:
#now, for each match, we want to average each specific stat for each home player
    #for match one, get all the home players' average overall rating, then average potential, etc.


for team in ["home", "away"]:
    for att in core_attributes:
        cols = [str(col).strip() for col in list(df.columns) if (f"{team}" in col) & (f"{att}" in col)]
        home_diffs[f"avg_{team}_player_{att}"] = df[cols].mean(axis = 1)
        if team == "away":
            home_diffs[f"home_diff_avg_player_{att}"] = home_diffs[f"avg_home_player_{att}"] - avg_player_stats[f"avg_away_player_{att}"]
            home_diffs = home_diffs.drop(columns = [f"avg_home_player_{att}", f"avg_away_player_{att}"])

In [2249]:
gk_atts[:5]

['home_1_gk_diving',
 'home_1_gk_handling',
 'home_1_gk_kicking',
 'home_1_gk_positioning',
 'home_1_gk_reflexes']

In [2250]:
for col in gk_atts[:5]:
    home_diffs[f"home_diff_{col[7:]}"] = (df[f"home_{col[5:]}"] - df[f"away_{col[5:]}"])/11

In [2251]:
for col in gk_atts[:5]:
    print(f"home_diff_{col[7:]}")
    print(f"home_{col[5:]}")
    print(f"away_{col[5:]}")

home_diff_gk_diving
home_1_gk_diving
away_1_gk_diving
home_diff_gk_handling
home_1_gk_handling
away_1_gk_handling
home_diff_gk_kicking
home_1_gk_kicking
away_1_gk_kicking
home_diff_gk_positioning
home_1_gk_positioning
away_1_gk_positioning
home_diff_gk_reflexes
home_1_gk_reflexes
away_1_gk_reflexes


In [2252]:
home_diffs.head(50)

,match_api_id,home_diff_avg_player_overall_rating,home_diff_avg_player_potential,home_diff_avg_player_crossing,home_diff_avg_player_finishing,home_diff_avg_player_heading_accuracy,home_diff_avg_player_short_passing,home_diff_avg_player_volleys,home_diff_avg_player_dribbling,home_diff_avg_player_curve,...,home_diff_avg_player_vision,home_diff_avg_player_penalties,home_diff_avg_player_marking,home_diff_avg_player_standing_tackle,home_diff_avg_player_sliding_tackle,home_diff_gk_diving,home_diff_gk_handling,home_diff_gk_kicking,home_diff_gk_positioning,home_diff_gk_reflexes
0,483134,-5.454545,-4.272727,-7.636364,-2.363636,1.545455,-3.818182,-7.636364,-10.272727,-10.272727,...,-10.818182,-15.636364,-3.000000,-2.818182,-1.272727,-7.090909,-5.727273,-0.636364,-5.636364,-5.727273
1,483133,8.454545,7.272727,12.090909,13.545455,7.727273,11.090909,12.909091,11.909091,7.181818,...,-0.272727,10.000000,2.818182,3.454545,1.818182,0.636364,-0.090909,0.454545,-0.454545,0.545455
2,489050,-6.636364,-4.545455,-4.272727,-5.363636,-7.272727,-9.636364,-11.363636,-1.909091,-13.181818,...,-8.272727,-11.818182,2.000000,1.909091,0.454545,-0.272727,-1.181818,-0.272727,-1.181818,-0.909091
3,489044,-6.181818,-3.727273,6.636364,-5.181818,-8.909091,-3.545455,-9.454545,3.727273,1.272727,...,-8.545455,-10.272727,-2.727273,-3.090909,-1.454545,-0.818182,0.090909,-0.545455,-0.636364,-0.272727
4,489043,6.818182,8.000000,6.909091,4.727273,6.000000,3.727273,10.545455,12.727273,14.727273,...,4.454545,2.000000,-0.090909,-1.000000,6.090909,-1.000000,-0.090909,0.363636,-0.454545,-0.090909
5,489049,7.818182,5.272727,7.636364,1.818182,4.363636,9.090909,12.272727,12.909091,8.000000,...,14.545455,4.181818,9.454545,14.454545,10.545455,0.090909,0.727273,0.727273,0.636364,0.636364
6,489989,-0.909091,-2.000000,-1.636364,-8.090909,0.000000,-1.181818,-15.090909,-5.727273,-3.909091,...,4.000000,-1.545455,3.818182,0.545455,0.727273,-0.636364,-0.454545,-0.545455,-0.727273,-0.727273
7,489051,6.818182,5.545455,13.545455,16.000000,-5.454545,10.363636,9.727273,8.272727,11.454545,...,9.181818,8.272727,1.363636,1.545455,9.181818,0.363636,1.000000,0.727273,1.272727,0.545455
8,483143,6.454545,7.272727,8.000000,8.272727,5.909091,11.909091,1.818182,0.545455,4.181818,...,8.909091,11.636364,-0.727273,-1.818182,-3.818182,0.363636,-0.181818,0.272727,0.727273,0.272727
9,499327,-9.727273,-6.545455,-5.727273,-13.454545,-9.000000,-9.090909,1.818182,-7.636364,2.454545,...,-3.636364,-13.727273,-6.818182,-3.636364,-6.636364,0.000000,-0.363636,-1.181818,-1.545455,-0.818182


In [2253]:
gk_atts[5:]

['away_1_gk_diving',
 'away_1_gk_handling',
 'away_1_gk_kicking',
 'away_1_gk_positioning',
 'away_1_gk_reflexes']

In [2254]:
home_diffs

,match_api_id,home_diff_avg_player_overall_rating,home_diff_avg_player_potential,home_diff_avg_player_crossing,home_diff_avg_player_finishing,home_diff_avg_player_heading_accuracy,home_diff_avg_player_short_passing,home_diff_avg_player_volleys,home_diff_avg_player_dribbling,home_diff_avg_player_curve,...,home_diff_avg_player_vision,home_diff_avg_player_penalties,home_diff_avg_player_marking,home_diff_avg_player_standing_tackle,home_diff_avg_player_sliding_tackle,home_diff_gk_diving,home_diff_gk_handling,home_diff_gk_kicking,home_diff_gk_positioning,home_diff_gk_reflexes
0,483134,-5.454545,-4.272727,-7.636364,-2.363636,1.545455,-3.818182,-7.636364,-10.272727,-10.272727,...,-10.818182,-15.636364,-3.000000,-2.818182,-1.272727,-7.090909,-5.727273,-0.636364,-5.636364,-5.727273
1,483133,8.454545,7.272727,12.090909,13.545455,7.727273,11.090909,12.909091,11.909091,7.181818,...,-0.272727,10.000000,2.818182,3.454545,1.818182,0.636364,-0.090909,0.454545,-0.454545,0.545455
2,489050,-6.636364,-4.545455,-4.272727,-5.363636,-7.272727,-9.636364,-11.363636,-1.909091,-13.181818,...,-8.272727,-11.818182,2.000000,1.909091,0.454545,-0.272727,-1.181818,-0.272727,-1.181818,-0.909091
3,489044,-6.181818,-3.727273,6.636364,-5.181818,-8.909091,-3.545455,-9.454545,3.727273,1.272727,...,-8.545455,-10.272727,-2.727273,-3.090909,-1.454545,-0.818182,0.090909,-0.545455,-0.636364,-0.272727
4,489043,6.818182,8.000000,6.909091,4.727273,6.000000,3.727273,10.545455,12.727273,14.727273,...,4.454545,2.000000,-0.090909,-1.000000,6.090909,-1.000000,-0.090909,0.363636,-0.454545,-0.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18846,1992225,3.454545,2.545455,1.181818,4.818182,2.181818,3.090909,3.636364,-3.454545,-4.363636,...,1.909091,1.727273,0.727273,3.363636,0.909091,-0.090909,0.272727,-0.272727,0.363636,-1.000000
18847,1992226,-3.818182,-3.000000,-10.545455,-10.363636,-6.818182,-7.727273,-4.363636,-6.727273,-13.272727,...,-3.636364,-3.454545,2.000000,1.727273,0.181818,-0.454545,0.909091,0.454545,-0.272727,0.181818
18848,1992227,-1.909091,-4.000000,1.727273,-4.727273,-4.000000,0.818182,-1.909091,-2.545455,-2.909091,...,-2.272727,-0.636364,-2.090909,-2.909091,-3.090909,0.000000,0.636364,0.000000,-0.454545,0.272727
18849,1992228,-8.181818,-8.272727,-6.818182,-7.363636,-3.545455,-7.363636,-5.000000,-6.545455,-8.545455,...,-9.727273,-7.090909,-10.909091,-7.636364,-5.636364,-0.636364,0.090909,-2.000000,-0.545455,-0.727273


In [2255]:
both_stats

,match_api_id,season,stage,match_date,Country,League,buildUpPlayPositioningClass,chanceCreationPositioningClass,defenceDefenderLineClass,away_buildUpPlayPositioningClass,...,buildUpPlaySpeed_home_diff,buildUpPlayDribbling_home_diff,buildUpPlayPassing_home_diff,chanceCreationPassing_home_diff,chanceCreationCrossing_home_diff,chanceCreationShooting_home_diff,defencePressure_home_diff,defenceAggression_home_diff,defenceTeamWidth_home_diff,Home_Score_Adv
0,684955,2009/2010,23,2010-02-22,Spain,Spain LIGA BBVA,Free Form,Organised,Offside Trap,Organised,...,0.0,NaN,-5.0,20.0,10.0,0.0,15.0,30.0,10.0,1
1,686171,2009/2010,20,2010-02-22,Portugal,Portugal Liga ZON Sagres,Organised,Organised,Offside Trap,Organised,...,0.0,NaN,0.0,5.0,-15.0,5.0,0.0,0.0,0.0,-1
2,659091,2009/2010,28,2010-02-23,England,England Premier League,Organised,Free Form,Cover,Organised,...,12.0,NaN,15.0,14.0,0.0,15.0,10.0,-20.0,10.0,3
3,704654,2009/2010,17,2010-02-24,Italy,Italy Serie A,Organised,Organised,Offside Trap,Organised,...,24.0,NaN,5.0,0.0,15.0,15.0,-5.0,-25.0,-5.0,1
4,704632,2009/2010,17,2010-02-24,Italy,Italy Serie A,Free Form,Free Form,Offside Trap,Free Form,...,22.0,NaN,0.0,-5.0,15.0,-10.0,15.0,-5.0,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19919,1992225,2015/2016,36,2016-05-25,Switzerland,Switzerland Super League,Organised,Organised,Cover,Organised,...,11.0,3.0,-8.0,14.0,12.0,8.0,11.0,15.0,6.0,-1
19920,1992226,2015/2016,36,2016-05-25,Switzerland,Switzerland Super League,NaN,NaN,NaN,Organised,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
19921,1992227,2015/2016,36,2016-05-25,Switzerland,Switzerland Super League,Organised,Organised,Cover,Organised,...,2.0,-6.0,14.0,-8.0,-10.0,13.0,3.0,6.0,-4.0,0
19922,1992228,2015/2016,36,2016-05-25,Switzerland,Switzerland Super League,Organised,Organised,Cover,Organised,...,6.0,3.0,-29.0,25.0,-30.0,20.0,3.0,11.0,3.0,-3


In [2256]:
combined = pd.merge(both_stats, home_diffs, on = "match_api_id")
combined

,match_api_id,season,stage,match_date,Country,League,buildUpPlayPositioningClass,chanceCreationPositioningClass,defenceDefenderLineClass,away_buildUpPlayPositioningClass,...,home_diff_avg_player_vision,home_diff_avg_player_penalties,home_diff_avg_player_marking,home_diff_avg_player_standing_tackle,home_diff_avg_player_sliding_tackle,home_diff_gk_diving,home_diff_gk_handling,home_diff_gk_kicking,home_diff_gk_positioning,home_diff_gk_reflexes
0,684955,2009/2010,23,2010-02-22,Spain,Spain LIGA BBVA,Free Form,Organised,Offside Trap,Organised,...,8.727273,9.181818,-1.636364,-4.909091,-0.636364,0.272727,-0.454545,-0.636364,1.181818,-0.545455
1,659091,2009/2010,28,2010-02-23,England,England Premier League,Organised,Free Form,Cover,Organised,...,9.545455,6.727273,7.272727,3.727273,2.909091,-0.272727,-0.272727,-0.090909,-0.909091,-0.727273
2,704654,2009/2010,17,2010-02-24,Italy,Italy Serie A,Organised,Organised,Offside Trap,Organised,...,0.909091,5.636364,6.181818,8.272727,1.727273,-0.363636,0.000000,0.090909,-0.181818,-0.636364
3,704632,2009/2010,17,2010-02-24,Italy,Italy Serie A,Free Form,Free Form,Offside Trap,Free Form,...,-3.000000,-4.363636,2.636364,3.090909,4.636364,0.636364,0.363636,-0.181818,0.636364,0.454545
4,674484,2009/2010,24,2010-02-27,Germany,Germany 1. Bundesliga,Organised,Organised,Cover,Organised,...,1.454545,2.454545,-3.454545,-3.909091,-6.181818,-0.545455,-0.090909,0.000000,0.090909,-0.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16974,1992225,2015/2016,36,2016-05-25,Switzerland,Switzerland Super League,Organised,Organised,Cover,Organised,...,1.909091,1.727273,0.727273,3.363636,0.909091,-0.090909,0.272727,-0.272727,0.363636,-1.000000
16975,1992226,2015/2016,36,2016-05-25,Switzerland,Switzerland Super League,NaN,NaN,NaN,Organised,...,-3.636364,-3.454545,2.000000,1.727273,0.181818,-0.454545,0.909091,0.454545,-0.272727,0.181818
16976,1992227,2015/2016,36,2016-05-25,Switzerland,Switzerland Super League,Organised,Organised,Cover,Organised,...,-2.272727,-0.636364,-2.090909,-2.909091,-3.090909,0.000000,0.636364,0.000000,-0.454545,0.272727
16977,1992228,2015/2016,36,2016-05-25,Switzerland,Switzerland Super League,Organised,Organised,Cover,Organised,...,-9.727273,-7.090909,-10.909091,-7.636364,-5.636364,-0.636364,0.090909,-2.000000,-0.545455,-0.727273


In [2257]:
path = DATA_PATH.joinpath("modeling_table.csv")
combined.to_csv(path, index = False)

In [2258]:
#I'm not including betting odds. These introduce data leakage and redundancy because they are based partly
#on player attributes and other features of the sort that are also used as explicit predictors. Also, they
#aren't consistently available for predicting matches far in advance, a goal of the model. Finally, they
#don't offer clear interpretability (such as we'll see when visualizing sample decision tree splits).

In [2259]:
pa.date.describe()

count                           183978
mean     2012-12-07 05:49:56.771353088
min                2007-02-22 00:00:00
25%                2011-02-22 00:00:00
50%                2013-09-20 00:00:00
75%                2014-12-19 00:00:00
max                2016-07-07 00:00:00
Name: date, dtype: object

In [2260]:
#matches
"""
count                            25979
mean     2012-06-30 17:53:53.334616320
min                2008-07-18 00:00:00
25%                2010-05-09 00:00:00
50%                2012-05-13 00:00:00
75%                2014-08-17 00:00:00
max                2016-05-25 00:00:00
Name: date, dtype: object
"""

'\ncount                            25979\nmean     2012-06-30 17:53:53.334616320\nmin                2008-07-18 00:00:00\n25%                2010-05-09 00:00:00\n50%                2012-05-13 00:00:00\n75%                2014-08-17 00:00:00\nmax                2016-05-25 00:00:00\nName: date, dtype: object\n'

In [2261]:
#%%sql
#df << SELECT * FROM
#Match m INNER JOIN Player_Attributes pa ON m.home_player_1 = pa.player_api_id;

In [2262]:
#df = pd.DataFrame(df)

In [2263]:
pa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   id                   183978 non-null  int64         
 1   player_fifa_api_id   183978 non-null  int64         
 2   player_api_id        183978 non-null  int64         
 3   date                 183978 non-null  datetime64[ns]
 4   overall_rating       183142 non-null  float64       
 5   potential            183142 non-null  float64       
 6   preferred_foot       183142 non-null  object        
 7   attacking_work_rate  180748 non-null  object        
 8   defensive_work_rate  183142 non-null  object        
 9   crossing             183142 non-null  float64       
 10  finishing            183142 non-null  float64       
 11  heading_accuracy     183142 non-null  float64       
 12  short_passing        183142 non-null  float64       
 13  volleys       